In [7]:
import pickle
import os
import json

import sys
from typing import List, Tuple, TypeVar, Dict, NoReturn, Iterable, Union

from pathlib import Path

import bots.rl18730

resources_file_path: Path = Path().cwd()

print(str(resources_file_path.parent/"rl18730.py"))

sys.path.append(str(resources_file_path.parent/"rl18730.py"))


from bots.rl18730 import PlayerRecord, GameRecordHistory, GameRecord, PlayerRecordNNEstimator,\
    RoleAllocationEnum, GamestateTree, TPlayer, gamerecord_history_unpickler

import bots.rl18730 as rl18730

print(rl18730.__dict__)

#sys.modules["GameRecordHistory"] = GameRecordHistory
#sys.modules["PlayerRecordNNEstimator"] = PlayerRecordNNEstimator
#sys.modules["RoleAllocationEnum"] = RoleAllocationEnum
#sys.modules["GamestateTree"] = GamestateTree
#sys.modules["TPlayer"] = TPlayer
import tensorflow as tf

from tensorflow import keras

from tensorflow.keras import models

import numpy as np




# https://datascience.stackexchange.com/questions/48796/how-to-feed-lstm-with-different-input-array-sizes

# https://keras.io/guides/understanding_masking_and_padding/

# TODO: load the PlayerRecordHolder pickle, use that data to train a model that can be put in PlayerRecordNNEstimator,
#   in order to estimate the likelihoods of each player being a spy.
#   validate the outputs of the model using the known spy info in the player records.
#   Remember to use the training set, test set, and validation set partitions present in PlayerRecordHolder,
#   to ensure that there's a good range of data being used.
# TODO: after that's done, save the model. Then, edit rl18730.py to load that model, and then use that model to
#   produce more refined player suspicion estimates.
# TODO: also compare the accuracy of the final neural network to the basic heuristic estimator already in PlayerRecord.
#   If the neural network turns out to be worse than the heuristic estimator, we don't bother with it.


M:\MSc\CE811\CE811_lowe_rachel_m\resistance_framework\bots\rl18730.py
{'__name__': 'bots.rl18730', '__doc__': None, '__package__': 'bots', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x00000130C05E4E50>, '__spec__': ModuleSpec(name='bots.rl18730', loader=<_frozen_importlib_external.SourceFileLoader object at 0x00000130C05E4E50>, origin='M:\\MSc\\CE811\\CE811_lowe_rachel_m\\resistance_framework\\bots\\rl18730.py'), '__file__': 'M:\\MSc\\CE811\\CE811_lowe_rachel_m\\resistance_framework\\bots\\rl18730.py', '__cached__': 'M:\\MSc\\CE811\\CE811_lowe_rachel_m\\resistance_framework\\bots\\__pycache__\\rl18730.cpython-39.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImport

TODO: load the PlayerRecordHolder pickle, use that data to train a model that can be put in PlayerRecordNNEstimator,
in order to estimate the likelihoods of each player being a spy.
validate the outputs of the model using the known spy info in the player records.
Remember to use the training set, test set, and validation set partitions present in PlayerRecordHolder,
to ensure that there's a good range of data being used.

TODO: after that's done, save the model. Then, edit rl18730.py to load that model,
  and then use that model to produce more refined player suspicion estimates.

TODO: also compare the accuracy of the final neural network to the basic heuristic estimator already in PlayerRecord.
  If the neural network turns out to be worse than the heuristic estimator, we don't bother with it.


NEURAL NETWORK STRUCTURE:

    Inputs:
        5 * ??? * 20
            5 players, ??? rounds, 20 wide per round
    Outputs:

In [8]:
import random


class json_gamerecordhistory_decoder(object):
    """
    only here because unpickling doesn't work
    """

    def __init__(self, json_strings: Iterable[str]) -> NoReturn:
        self._game_records: List[Dict[str, Union[Tuple[float, ...], Tuple[int, ...], Tuple[int, ...]]]] = []
        for j in json_strings:
            d = json.loads(j)
            self._game_records.append(d)

    @property
    def get_records(self) -> List[Dict[str, Union[Tuple[float, ...], Tuple[int, ...], Tuple[int, ...]]]]:
        """
        Obtains all of the records held in this PlayerRecordHolder
        :return: the list of all the records in this holder (arranged per game)
        """
        return self._game_records.copy()

    def get_training_set_and_test_set_and_validation_set(
            self, training_size: float = 0.4, test_size: float = 0.4
    ) -> Tuple[
        List[Dict[str, Union[Tuple[float, ...], Tuple[int, ...], Tuple[int, ...]]]],
        List[Dict[str, Union[Tuple[float, ...], Tuple[int, ...], Tuple[int, ...]]]],
        List[Dict[str, Union[Tuple[float, ...], Tuple[int, ...], Tuple[int, ...]]]],
    ]:
        """
        Attempts to create a training set and a validation set from the data we have.
        :param training_size: what proportion of our data are we putting into our training set?
        :param test_size: what proportion of our data are we putting into our test set?
        :return: a tuple with (training set data, test set data, validation set data)
        """

        rec_len: int = len(self._game_records)

        training_len: int = int(rec_len * training_size)
        test_len: int = int(rec_len * test_size)

        tt_len: int = training_len + test_len

        val_len: int = rec_len - tt_len

        assert training_len > 0
        assert test_len > 0
        assert val_len > 0

        shuffled_all: List[Dict[str, Tuple[float, ...], Tuple[int, ...], Tuple[int, ...]]] = \
            random.sample(self._game_records, rec_len)

        return shuffled_all[0:training_len], shuffled_all[training_len:tt_len], shuffled_all[tt_len:rec_len]

log_file = open("heuristic bayes training data log.log","r")

grh_dec = json_gamerecordhistory_decoder(log_file)
log_file.close()

data_sets = grh_dec.get_training_set_and_test_set_and_validation_set()

print(len(data_sets[0]))
print(len(data_sets[1]))
print(len(data_sets[2]))

1806
1806
903


In [9]:
# TODO: make and train a neural network, compare outputs of that
# to the heuristic suspicion outputs